In [216]:
!pip install pydicom opencv-python

In [217]:
import os
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import cv2
from glob import glob

In [218]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [219]:
input_folder = '/content/drive/MyDrive/Colab_Notebooks/I831065'
output_folder = "/content/drive/MyDrive/Colab_Notebooks/manual_cropped"
os.makedirs(output_folder, exist_ok=True)

In [220]:
# Загружаем и сортируем DICOM-файлы по X (сагиттальная ось)
dcm_files = glob(os.path.join(input_folder, "*.dcm"))
slices = [pydicom.dcmread(f) for f in dcm_files]

In [221]:
# Убедимся что координаты есть
slices = [s for s in slices if 'ImagePositionPatient' in s]
slices.sort(key=lambda s: float(s.ImagePositionPatient[0]))  # сортировка по x

In [222]:
# Собираем volume как (x, y, z) -> потом транспонируем
volume = np.stack([s.pixel_array for s in slices])  # shape = (x, y, z)
volume = (volume - volume.min()) / (volume.max() - volume.min()) * 255
volume = volume.astype(np.uint8)

In [223]:
# Обрезаем крайние срезы
trim = 35
volume_trimmed = volume[trim:len(volume) - trim]
center_index = len(volume_trimmed) // 2

In [224]:
# Задаем вручную параметры:
base_y, base_h = 40, 160
base_x, base_w = 30, 180
max_increase = 1

# Желаемые размеры для изображений (Взял 256x256)
target_size = (256, 256)


In [225]:
# Применяем обрезку и собираем numpy массив
cropped_images = []

for i in range(len(volume_trimmed)):
    slice_img = volume_trimmed[i]
    dist = abs(i - center_index)
    factor = 1 - dist / center_index
    delta = int(max_increase * factor)

    y_start = base_y - delta
    y_end = base_y + base_h + delta
    x_start = base_x - delta
    x_end = base_x + base_w + delta

    # Обрезка изображения
    cropped = slice_img[max(y_start, 0):min(y_end, slice_img.shape[0]),
                        max(x_start, 0):min(x_end, slice_img.shape[1])]

    # Эти две строчки для сохранения и после отображения результата
    save_path = os.path.join(output_folder, f"cropped_{i:03d}.png")
    cv2.imwrite(save_path, cropped)

    # Изменение размера изображения до target_size
    cropped_resized = cv2.resize(cropped, target_size)

    # Добавление в массив
    cropped_images.append(cropped_resized)

In [226]:
# Преобразуем список в numpy массив
cropped_data = np.array(cropped_images)
# Преобразуем в DataFrame
df = pd.DataFrame(cropped_data.reshape(cropped_data.shape[0], -1))

In [227]:
files = sorted(os.listdir(output_folder))
step = 10

In [228]:
for i in range(0, len(files), step):
    plt.figure(figsize=(15, 5))
    for j in range(step):
        idx = i + j
        if idx >= len(files):
            break
        img_path = os.path.join(output_folder, files[idx])
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        plt.subplot(1, step, j + 1)
        plt.imshow(img, cmap='gray')
        plt.title(f"Slice {idx + trim}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [229]:
import pickle
import pandas as pd
from google.colab import files
# сериализуем массив
packed = pickle.dumps(cropped_data)

# От балды ID пациента и тяжесть болезни
subject_id = 'subject_001'
severity = 2

df = pd.DataFrame([{
    'id': subject_id,
    'severity': severity,
    'image_data': packed
}])

In [230]:
# Сериализуем, тут получше вышло
df.to_pickle("dataset.pkl")
files.download('dataset.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [231]:
# Самый наилучший вариант как по мне, из 42 мб изначального файла вышло 4
df.to_parquet("dataset.parquet", compression="brotli")
files.download('dataset.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>